In [ ]:
!unzip Train_Dataset.zip

In [ ]:
!rm Train_Dataset.zip

### Import Libraries

In [1]:
#importing neccesary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### Loading Dataset to Dataframe

In [2]:
dataset = pd.read_csv('train.csv',low_memory=False)
dataset.dtypes

ID                           object
Customer_ID                  object
Month                        object
Name                         object
Age                          object
SSN                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance              object
Credit_Score                

In [3]:

#dropping features which are not useful for determining the credit score
final_dataset = dataset.drop(["Amount_invested_monthly", "SSN", "Name", "Month", "Age", "Num_Bank_Accounts", "Num_Credit_Card", "Interest_Rate", "Type_of_Loan", "Changed_Credit_Limit", "Num_Credit_Inquiries", "Credit_Mix", "Credit_Utilization_Ratio"], axis=1)

In [4]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ID                      100000 non-null  object 
 1   Customer_ID             100000 non-null  object 
 2   Occupation              100000 non-null  object 
 3   Annual_Income           100000 non-null  object 
 4   Monthly_Inhand_Salary   84998 non-null   float64
 5   Num_of_Loan             100000 non-null  object 
 6   Delay_from_due_date     100000 non-null  int64  
 7   Num_of_Delayed_Payment  92998 non-null   object 
 8   Outstanding_Debt        100000 non-null  object 
 9   Credit_History_Age      90970 non-null   object 
 10  Payment_of_Min_Amount   100000 non-null  object 
 11  Total_EMI_per_month     100000 non-null  float64
 12  Payment_Behaviour       100000 non-null  object 
 13  Monthly_Balance         98800 non-null   object 
 14  Credit_Score         

In [5]:
#dataset.to_csv('final_dataset.csv', index=False)  
#this is the mistake you are converting the initial dataframe (dataset) itself to csv
#instead of the new dataframe (final dataset)

final_dataset.to_csv('final_dataset.csv', index=False)

In [6]:
#final_dataset = pd.read_csv('final_dataset.csv', low_memory=False)
x = final_dataset.iloc[:,:-1].values
y = final_dataset.iloc[:,-1].values

In [7]:
x

array([['0x1602', 'CUS_0xd40', 'Scientist', ..., 49.57494921489417,
        'High_spent_Small_value_payments', '312.49408867943663'],
       ['0x1603', 'CUS_0xd40', 'Scientist', ..., 49.57494921489417,
        'Low_spent_Large_value_payments', '284.62916249607184'],
       ['0x1604', 'CUS_0xd40', 'Scientist', ..., 49.57494921489417,
        'Low_spent_Medium_value_payments', '331.2098628537912'],
       ...,
       ['0x25feb', 'CUS_0x942c', 'Mechanic', ..., 35.10402261040746,
        'High_spent_Large_value_payments', '516.8090832742814'],
       ['0x25fec', 'CUS_0x942c', 'Mechanic', ..., 35.10402261040746,
        'Low_spent_Large_value_payments', '319.1649785257098'],
       ['0x25fed', 'CUS_0x942c', 'Mechanic', ..., 35.10402261040746,
        '!@9#%8', '393.6736955618808']], dtype=object)

In [8]:
y

array(['Good', 'Good', 'Good', ..., 'Poor', 'Standard', 'Poor'],
      dtype=object)

In [9]:
final_dataset.head()

,ID,Customer_ID,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Outstanding_Debt,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,Scientist,19114.12,1824.843333,4,3,7,809.98,22 Years and 1 Months,No,49.574949,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,Scientist,19114.12,NaN,4,-1,NaN,809.98,NaN,No,49.574949,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,Scientist,19114.12,NaN,4,3,7,809.98,22 Years and 3 Months,No,49.574949,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,Scientist,19114.12,NaN,4,5,4,809.98,22 Years and 4 Months,No,49.574949,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,Scientist,19114.12,1824.843333,4,6,NaN,809.98,22 Years and 5 Months,No,49.574949,High_spent_Medium_value_payments,341.48923103222177,Good


In [10]:
final_dataset.shape

(100000, 15)

## Data Cleaning and fixing variables

In [11]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ID                      100000 non-null  object 
 1   Customer_ID             100000 non-null  object 
 2   Occupation              100000 non-null  object 
 3   Annual_Income           100000 non-null  object 
 4   Monthly_Inhand_Salary   84998 non-null   float64
 5   Num_of_Loan             100000 non-null  object 
 6   Delay_from_due_date     100000 non-null  int64  
 7   Num_of_Delayed_Payment  92998 non-null   object 
 8   Outstanding_Debt        100000 non-null  object 
 9   Credit_History_Age      90970 non-null   object 
 10  Payment_of_Min_Amount   100000 non-null  object 
 11  Total_EMI_per_month     100000 non-null  float64
 12  Payment_Behaviour       100000 non-null  object 
 13  Monthly_Balance         98800 non-null   object 
 14  Credit_Score         

In [29]:
100000-84998

15002

#### Identifying the number of Null Values and related Customer Id for Validation

In [12]:
null_mask = final_dataset["Monthly_Inhand_Salary"].isnull()
null_mask
final_dataset[null_mask]

,ID,Customer_ID,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Outstanding_Debt,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Payment_Behaviour,Monthly_Balance,Credit_Score
1,0x1603,CUS_0xd40,Scientist,19114.12,NaN,4,-1,NaN,809.98,NaN,No,49.574949,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,Scientist,19114.12,NaN,4,3,7,809.98,22 Years and 3 Months,No,49.574949,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,Scientist,19114.12,NaN,4,5,4,809.98,22 Years and 4 Months,No,49.574949,Low_spent_Small_value_payments,223.45130972736786,Good
5,0x1607,CUS_0xd40,Scientist,19114.12,NaN,4,8,4,809.98,22 Years and 6 Months,No,49.574949,!@9#%8,340.4792117872438,Good
11,0x1611,CUS_0x21b1,Teacher,34847.84,NaN,1,3,3_,605.03,26 Years and 10 Months,No,18.816215,Low_spent_Medium_value_payments,465.6762241330048,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99944,0x25f9e,CUS_0x51b3,Media_Manager,59146.36,NaN,1,8,6,418.03,NaN,No,26.778419,Low_spent_Small_value_payments,147.67137163088418,Standard
99955,0x25fad,CUS_0x2084,Architect,38321.39,NaN,4,11,7,678.57,17 Years and 2 Months,No,362.072453,Low_spent_Large_value_payments,197.46057710771268,Standard
99963,0x25fb9,CUS_0x372c,Lawyer,42903.79,NaN,1,14,0,1079.48,NaN,No,34.975457,High_spent_Large_value_payments,520.6622073726148,Standard
99975,0x25fc9,CUS_0xf16,Media_Manager,16680.35,NaN,4,1,0,897.16,NaN,No,41.113561,Low_spent_Large_value_payments,310.8838062920152,Good


In [14]:
grouped_df = final_dataset.groupby("Customer_ID")

In [19]:
print(grouped_df["Monthly_Inhand_Salary"].count())

Customer_ID
CUS_0x1000    7
CUS_0x1009    8
CUS_0x100b    7
CUS_0x1011    7
CUS_0x1013    5
             ..
CUS_0xff3     7
CUS_0xff4     7
CUS_0xff6     8
CUS_0xffc     8
CUS_0xffd     7
Name: Monthly_Inhand_Salary, Length: 12500, dtype: int64


**Observation**
* Totally 15002 entries has null values for the Monthly_Inhand_Salary
* Out of 100000 , we have 12500 Unique Customer_ID
* Need to impute the Missing Values for the 15002 Entries

### Deal with the missing values of Monthly_Inhand_Salary columns

In [20]:
# Convert the Customer_ID column to a factor
final_dataset['Customer_ID'] = final_dataset['Customer_ID'].astype('category')

# Calculate the median Inhand Salary by customer using groupby and summarize
median_Inhand_salary_by_customer = final_dataset.groupby('Customer_ID').agg(median_salary=('Monthly_Inhand_Salary', 'median'))

# Join the median salary column to the dataset
final_dataset = pd.merge(final_dataset, median_Inhand_salary_by_customer, on='Customer_ID', how='left')

# Replace missing values in Monthly Inhand Salary column with median salary
final_dataset['Monthly_Inhand_Salary'].fillna(final_dataset['median_salary'], inplace=True)

# Drop the median salary column
final_dataset.drop('median_salary', axis=1, inplace=True)


In [21]:
median_Inhand_salary_by_customer

,median_salary
Customer_ID,
CUS_0x1000,2706.161667
CUS_0x1009,4250.390000
CUS_0x100b,9549.782500
CUS_0x1011,5208.872500
CUS_0x1013,7962.415000
...,...
CUS_0xff3,1176.398750
CUS_0xff4,2415.855000
CUS_0xff6,9727.326667


In [22]:
median_Inhand_salary_by_customer.info()

<class 'pandas.core.frame.DataFrame'>
CategoricalIndex: 12500 entries, CUS_0x1000 to CUS_0xffd
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   median_salary  12500 non-null  float64
dtypes: float64(1)
memory usage: 477.8 KB


In [23]:
median_Inhand_salary_by_customer.describe()

,median_salary
count,12500.000000
mean,4198.390362
std,3187.681114
min,303.645417
25%,1626.719792
50%,3094.825000
75%,5961.664375
max,15204.633333


In [32]:
final_dataset["Monthly_Inhand_Salary"].describe()

count    100000.000000
mean       4198.490149
std        3187.491752
min         303.645417
25%        1626.761667
50%        3095.978333
75%        5961.637500
max       15204.633333
Name: Monthly_Inhand_Salary, dtype: float64

In [24]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   ID                      100000 non-null  object  
 1   Customer_ID             100000 non-null  category
 2   Occupation              100000 non-null  object  
 3   Annual_Income           100000 non-null  object  
 4   Monthly_Inhand_Salary   100000 non-null  float64 
 5   Num_of_Loan             100000 non-null  object  
 6   Delay_from_due_date     100000 non-null  int64   
 7   Num_of_Delayed_Payment  92998 non-null   object  
 8   Outstanding_Debt        100000 non-null  object  
 9   Credit_History_Age      90970 non-null   object  
 10  Payment_of_Min_Amount   100000 non-null  object  
 11  Total_EMI_per_month     100000 non-null  float64 
 12  Payment_Behaviour       100000 non-null  object  
 13  Monthly_Balance         98800 non-null   object  
 14  Credi

### Deal with the missing values and '_' of Num_of_Delayed_Payment 

In [31]:
null_mask_1 = final_dataset["Num_of_Delayed_Payment"].isnull()
null_mask_1
final_dataset[null_mask_1]

,ID,Customer_ID,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Outstanding_Debt,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Payment_Behaviour,Monthly_Balance,Credit_Score
1,0x1603,CUS_0xd40,Scientist,19114.12,1824.843333,4,-1,NaN,809.98,NaN,No,49.574949,Low_spent_Large_value_payments,284.62916249607184,Good
4,0x1606,CUS_0xd40,Scientist,19114.12,1824.843333,4,6,NaN,809.98,22 Years and 5 Months,No,49.574949,High_spent_Medium_value_payments,341.48923103222177,Good
30,0x162c,CUS_0xb891,Entrepreneur,30689.89,2612.490833,1,5,NaN,632.46,17 Years and 9 Months,No,16.415452,Low_spent_Large_value_payments,443.5499567906391,Standard
32,0x1632,CUS_0x1cdb,Developer,35547.71_,2853.309167,0,5,NaN,943.86,30 Years and 8 Months,Yes,0.000000,!@9#%8,288.60552234930395,Standard
33,0x1633,CUS_0x1cdb,Developer,35547.71,2853.309167,0,9,NaN,943.86,30 Years and 9 Months,NM,0.000000,High_spent_Medium_value_payments,460.88727561667037,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99973,0x25fc7,CUS_0xf16,Media_Manager,16680.35,1528.029167,4,1,NaN,897.16,21 Years and 6 Months,No,41.113561,Low_spent_Small_value_payments,233.3015390325111,Good
99974,0x25fc8,CUS_0xf16,Media_Manager,16680.35_,1528.029167,4,1,NaN,897.16,21 Years and 7 Months,No,41.113561,Low_spent_Small_value_payments,340.68483402219437,Good
99992,0x25fe6,CUS_0x942c,Mechanic,39628.99,3359.415833,2,23,NaN,502.38,31 Years and 3 Months,No,35.104023,Low_spent_Small_value_payments,189.64108011929028,Poor
99993,0x25fe7,CUS_0x942c,Mechanic,39628.99_,3359.415833,2,23,NaN,502.38,31 Years and 4 Months,No,58638.000000,Low_spent_Medium_value_payments,400.1044655284761,Standard


In [33]:
final_dataset["Num_of_Delayed_Payment"].describe()

count     92998
unique      749
top          19
freq       5327
Name: Num_of_Delayed_Payment, dtype: object

In [34]:
# replace the '_' with ''
final_dataset['Num_of_Delayed_Payment'] = final_dataset['Num_of_Delayed_Payment'].replace('_', '', regex=True)

# convert to numeric
final_dataset['Num_of_Delayed_Payment'] = final_dataset['Num_of_Delayed_Payment'].astype(float)

In [35]:
final_dataset["Num_of_Delayed_Payment"].describe()

count    92998.000000
mean        30.923342
std        226.031892
min         -3.000000
25%          9.000000
50%         14.000000
75%         18.000000
max       4397.000000
Name: Num_of_Delayed_Payment, dtype: float64

**Observation**

* Totally 7002 entries has null values for the Num_of_Delayed_Payment
* We could see Negative Values ()
* Need to impute the Missing Values for the 7002 Entries

In [ ]:
# Calculate the median delayed payments by customer using groupby and summarize
median_delayed_by_customer = final_dataset.groupby('Customer_ID').agg(median_delayed=('Num_of_Delayed_Payment', 'median'))

# Join the median delayed payments column to the dataset
final_dataset = pd.merge(final_dataset, median_delayed_by_customer, on='Customer_ID', how='left')

# Replace missing values in Num of Delayed Payment column with median delayed payments
final_dataset['Num_of_Delayed_Payment'].fillna(final_dataset['median_delayed'], inplace=True)


# Drop the median delayed payments column
final_dataset.drop('median_delayed', axis=1, inplace=True)

In [ ]:
final_dataset.info()

### Deal with the missing values and '_' of Monthly_Balance

In [ ]:
# replace the '_' with ''
final_dataset['Monthly_Balance'] = final_dataset['Monthly_Balance'].replace('_', '', regex=True)
# convert to numeric
final_dataset['Monthly_Balance'] = final_dataset['Monthly_Balance'].astype(float)

In [ ]:
# Calculate the median delayed payments by customer using groupby and summarize
median_Monthly_Balance_customer = final_dataset.groupby('Customer_ID').agg(median_Monthly_Balance=('Monthly_Balance', 'median'))

# Join the median delayed payments column to the dataset
final_dataset = pd.merge(final_dataset, median_Monthly_Balance_customer, on='Customer_ID', how='left')

# Replace missing values in Num of Delayed Payment column with median delayed payments
final_dataset['Monthly_Balance'].fillna(final_dataset['median_Monthly_Balance'], inplace=True)

# Drop the median delayed payments column
final_dataset.drop('median_Monthly_Balance', axis=1, inplace=True)

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.info()

### Dealing with the missing value in Credit_History_Age column

In [ ]:
# Define function to convert string to number of months
def convert_to_months(s):
    # If value is null or empty, return default value 0
    if pd.isna(s) or s.strip() == "":
        return 0
    # Split string into years and months
    years, months = s.split(" Years and ")
    months = months.replace(" Months", "")
    # Convert to total months
    total_months = int(years) * 12 + int(months)
    return total_months

# Apply the function to the "age" column and create a new column "age_in_months"
final_dataset["Credit_History_Age"] = final_dataset["Credit_History_Age"].apply(convert_to_months)

In [ ]:
# Calculate the median credit history age by customer using groupby and summarize
median_credit_history_age_customer = final_dataset.groupby('Customer_ID').agg(median_credit_history_age=('Credit_History_Age', 'median'))

# Join the median credit history age column to the dataset
final_dataset = pd.merge(final_dataset, median_credit_history_age_customer, on='Customer_ID', how='left')

# Replace rows with a value of 0 in Credit History Age column with median credit history age
final_dataset['Credit_History_Age'] = final_dataset['Credit_History_Age'].apply(lambda x: final_dataset['median_credit_history_age'][x] if x == 0 else x)

# Drop the median credit history age column
final_dataset.drop('median_credit_history_age', axis=1, inplace=True)

In [ ]:
final_dataset.info()

### Dealing with Occupation column

In [ ]:
final_dataset['Occupation'].value_counts()

In [ ]:
# Remove rows where the "Occupation" column has the value "_______"
final_dataset = final_dataset[~(final_dataset['Occupation'] == '_______')]
final_dataset['Occupation'].value_counts()

### Dealing with Annual Income column

In [ ]:
final_dataset['Annual_Income'].value_counts()

In [ ]:
# Remove rows where the "Annual_Income" column has the value "_"
final_dataset['Annual_Income'] = final_dataset['Annual_Income'].str.replace('_', '')
# Converting "Annual_Income" column to numeric
#final_dataset['Annual_Income'] = pd.to_(final_dataset['Annual_Income'],errors='coerce')
final_dataset['Annual_Income'].value_counts()


In [ ]:
# Adjust incorrect data entries
sym = '\\`*_{}[]()>#@+!$:;'
col_int = ['Annual_Income','Delay_from_due_date','Num_of_Delayed_Payment',
           'Outstanding_Debt','Total_EMI_per_month', 'Monthly_Balance']

col_str = ['Occupation', 'Credit_History_Age','Payment_of_Min_Amount','Credit_Score']

for i in col_int:
    for c in sym:
        final_dataset[i] = final_dataset[i].astype(str).str.replace(c,'')
        
for i in col_str:
    for c in sym:
        final_dataset[i] = final_dataset[i].replace(c,'')

### Dealing with Credit Score column

In [ ]:
final_dataset['Credit_Score'].value_counts()

### Dealing with Payment Behaviour column 

In [ ]:
final_dataset['Payment_Behaviour'].value_counts()

In [ ]:
# Remove rows where the "Payment_Behaviour" column has the value "!@9#%8"
final_dataset = final_dataset[~(final_dataset['Payment_Behaviour'] == '!@9#%8')]

In [ ]:
final_dataset['Payment_Behaviour'].value_counts()

In [ ]:
final_dataset["Payment_Behaviour"] = final_dataset["Payment_Behaviour"].replace({"High_spent_Large_value_payments": 6,
                                                                       "High_spent_Medium_value_payments": 5,
                                                                       "High_spent_Small_value_payments": 4,
                                                                       "Low_spent_Large_value_payments": 3,
                                                                       "Low_spent_Medium_value_payments": 2,
                                                                       "Low_spent_Small_value_payments": 1})


In [ ]:
final_dataset["Payment_Behaviour"].value_counts()

Dealing with Total_EMI_per_month column

In [ ]:
final_dataset['Total_EMI_per_month'].value_counts()

Dealing with the Payment_of_Min_Amount column

In [ ]:
final_dataset['Payment_of_Min_Amount'].value_counts()

In [ ]:
final_dataset['Payment_of_Min_Amount'] = final_dataset['Payment_of_Min_Amount'].replace({"No": 2,
                                                                       "Yes": 1,
                                                                       "NM": 0})

In [ ]:
final_dataset['Payment_of_Min_Amount'].value_counts()

Dealing with Outstanding_Debt column

In [ ]:
final_dataset['Outstanding_Debt'].value_counts()

In [ ]:
# Remove rows where the "Outstanding_Debt" column has the value "_"
#final_dataset['Outstanding_Debt'] = final_dataset['Outstanding_Debt'].str.replace('_', '')
# Converting "Outstanding_Debt" column to numeric
#final_dataset['Outstanding_Debt'] = pd.to_numeric(final_dataset['Outstanding_Debt'],errors='coerce')


int_col = ['Delay_from_due_date','Num_of_Delayed_Payment','Outstanding_Debt',
           'Total_EMI_per_month','Monthly_Balance','Payment_Behaviour','Credit_History_Age','Annual_Income']

for i in int_col:
    final_dataset[i] = final_dataset[i].astype(float)

In [ ]:
final_dataset['Outstanding_Debt'].value_counts()

Dealing with Delay_from_due_date column

In [ ]:
final_dataset['Delay_from_due_date'].value_counts()

In [ ]:
final_dataset.head()

In [ ]:
final_dataset['Delay_from_due_date'].value_counts()

Dealing with the Num_of_Loan column

In [ ]:
final_dataset['Num_of_Loan'].value_counts()

In [ ]:
final_dataset.loc[final_dataset['Num_of_Loan'] == "-100", 'Num_of_Loan'] = np.nan

In [ ]:
# Calculate the median Num_of_Loan by customer using groupby and summarize
#median_Num_of_Loan_customer = final_dataset.groupby('Customer_ID').agg(median_Num_of_Loan=('Num_of_Loan', 'median'))

# Join the median Num_of_Loan column to the dataset
final_dataset = pd.merge(final_dataset, median_Num_of_Loan_customer, on='Customer_ID', how='left')

# Replace rows with a value of 0 in Num_of_Loan column with median credit history age
final_dataset['Num_of_Loan'].fillna(final_dataset['median_Num_of_Loan'], inplace=True)

# Drop the median Num_of_Loan column
final_dataset.drop('median_Num_of_Loan', axis=1, inplace=True)


In [ ]:
final_dataset.head()

In [ ]:
#Using the Inter Quantile Range to drop outliers
Q1 = final_dataset.Annual_Income.quantile(0.25)
Q3 = final_dataset.Annual_Income.quantile(0.75)

IQR = Q3 - Q1
final_dataset = final_dataset.drop(final_dataset.loc[final_dataset['Annual_Income'] > (Q3 + 1.5 * IQR)].index)
final_dataset = final_dataset.drop(final_dataset.loc[final_dataset['Annual_Income'] < (Q1-1.5*IQR)].index)

In [ ]:
sns.boxplot(x=final_dataset['Annual_Income'])

In [ ]:
sns.boxplot(x=final_dataset['Credit_History_Age'])

In [ ]:
sns.boxplot(x=final_dataset['Outstanding_Debt'])

In [ ]:
#Occupation count
counting_occupation = final_dataset['Occupation'].value_counts(dropna=False)


In [ ]:
#Bar graph for occupations
sns.set(rc={'figure.figsize':(20,10)})
sns.barplot(x=counting_occupation)
plt.title('Bar graph for occupations', fontsize=16)
plt.ylabel('Count', fontsize=14)
plt.xlabel('Occupation', fontsize=14)
plt.xticks(rotation=45)
plt.show()

Lets work on the outliers in different columns like "Num_of_Delayed_Payment", "Total_EMI_per_month", "Num_of_Loan" and "Annual_Income"

In [ ]:
#sorted(final_dataset['Num_of_Delayed_Payment'])

In [ ]:
#quantile1, quantile3 = np.percentile(final_dataset['Num_of_Delayed_Payment'],[25,75])

In [ ]:
#print(quantile1,quantile3)

In [ ]:
#iqr_value = quantile3 - quantile1
#print(iqr_value)

In [ ]:
#lowerbound = quantile1 - (1.5 * iqr_value)
#upperbound = quantile3 + (1.5 * iqr_value)
#print(lowerbound,upperbound)